In [1]:
# 라이브러리 임포트
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# XOR 데이터  양수(1) 음수(0) 일경우에는 양수(1)고 나머지는 음수(0)
LABELS = [0,0,1,1] 
CENTERS = [(-3,-3),(3,3),(-3,3),(3,-3)]

In [3]:
class MultilayerPerceptron(nn.Module):
    """
    가중치 초기화 
    input_size : 입력 크기
    hidden_size : 은닉층 크기
    output_size : 출력 크기
    num_hidden_layers : 은닉층 개수
    hidden_activation : 활성화 함수
    """
    def __init__(self, input_size, hidden_size=2, output_size=3, num_hidden_layers=1, hidden_activation=nn.Sigmoid):
        super(MultilayerPerceptron, self).__init__
        self.module_list = nn.ModuleList()
        interim_input_size =  input_size
        interim_output_size = hidden_size
        for _ in range(num_hidden_layers):
            self.module_list.append(nn.Linear(interim_input_size,interim_output_size))
            self.module_list.append(hidden_activation())
            interim_input_size = interim_output_size
        self.fc_final = nn.Linear(interim_input_size,  output_size)  
        self.last_forward_cache = []
    # MLP 정방향 계산
    def forward(self, x, apply_softmax = False):  # 크로스 엔트로피를 사용하려면 apply_softmax 가 false
        self.last_forward_cache = []
        self.last_forward_cache.append(x.to('cpu').numpy())
        for module in self.module_list:
            x = module(x)
            self.last_forward_cache.append(x.to('cpu').numpy())
        output = self.fc_final(x)
        self.last_forward_cache.append(x.to('cpu').numpy())
        if apply_softmax:
            output = F.softmax(output, dim=1)
        return output

In [4]:
# 데이터 생성함수
def get_data(batch_size):
    x_data = []
    y_target = np.zeros(batch_size)
    n_center = len(CENTERS)
    for batch_id in range(batch_size):
        center_idx = np.random.randint(0,n_center)
        x_data.append(np.random.normal(loc = CENTER[center_idx]))
        y_targets[batch_i] = LABELS[center_idx]
    return torch.tensor(x_data,dtype=torch.float32), torch.tensor(y_target, dtype=torch.int16)

In [ ]:
# 결과 시각화 함수
def visulize_results(perceptron, x_data,y_truth,n_samples=1000, ax=None, epoch=None,
                     title='',levels=[0.3,0.4,0.5], linestyles=['--','-','--']):
    _, y_pred =  perceptron(x_data,apply_softmax=True).max(dim=1)
    y_pred = y_pred.data.numpy()
    
    x_data = x_data.data.numpy()
    y_truth = y_truth.data.numpy()
    
    n_classess = len(set(LABELS))
    
    all_x =[ []  for _ in range(n_classess)]
    all_colors =[ []  for _ in range(n_classess)]
    
    colors = ['orange','green']
    markers = ['o', '*']
    enge_color = {'o':'orange', '*':'green'}
    
    for x_list, color_list,marker in zip(all_x, all_color,markers):
        ax.scatter(x_list[:,0],x_list[:,1], edgecolor=enge_color[marker], marker=marker, facecolor=color_list,s=100)
    xlim = (
        min( [ x_list[:,0].min() for x_list in all_x  ]  ),
        max( [ x_list[:,0].max() for x_list in all_x  ]  )
    )
    ylim = (
        min( [ x_list[:,1].min() for x_list in all_x  ]  ),
        max( [ x_list[:,1].max() for x_list in all_x  ]  )
    )
    # 평면
    xx = np.linespace(xlim[0], xlim[1], 30)
    yy = np.linespace(xlim[0], xlim[1], 30)
    YY, XX = np.meshgrid(yy,xx)
    xy = np.vstack([XX.ravel(), YY.ravel()] ).T
    
    for i in range(n_classess):
        Z = perceptron(torch.tensor(xy, dtype=torch.float32), apply_softmax =True)
        Z = Z[:,i].data.numpy().reshape(XX.shape)
        ax.coutour(XX,YY, Z, colors=colors[i], levels=levels, linestyles = linestyles)
    # 부가적인 출력
    plt.subplot(title)
    if epoch is not None:
        plt.text(xlim[0], ylim[1], f'Epoch = {str(}'
    

In [10]:
all_color = ['orange','green']
markers = ['o', '*']
enge_color = {'o':'orange', '*':'green'}
all_x = [[1,2,3],[4,5,6]]
for x_list, color_list,marker in zip(all_x, all_color,markers):
    print(x_list,color_list,marker)

[1, 2, 3] orange o
[4, 5, 6] green *
